# INDIVIDUAL CLUSTERING

In [1]:
import numpy as np
import datetime
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt

### Detalles

### Lista de Clientes

In [2]:
# Extraemos la lista de clientes sin repetir

def leer_data():
    outfile='./data/consulta.csv'
    data = pd.read_csv(outfile)
    return data

data = leer_data()
clientes =  data.groupby('client_id').client_id.count().index

## Temporal TXs footprint

In [3]:
# Definimos la ruta de los archivos 

file='U'
raw_data='./data/%s.json' %(file)
individual_footprint="%s.individual_footprint" %(raw_data)
individual_clusters="%s.clusters" %(individual_footprint)
individual_labels="%s.labels" %(individual_footprint)

### Funciones

In [4]:
def process_footprint(data,tests,log=False):
    from sklearn.cluster import MiniBatchKMeans
    #KMeans(init='k-means++', n_clusters=k, n_init=10)
    import datetime
    K={}
    for k in tests:
        if k<=len(data):
            if log:
                print("%s: processing %s"%(datetime.datetime.now(),k))
            K[k]=bench_k_means(MiniBatchKMeans(init='k-means++', n_clusters=k, batch_size=100,
                      n_init=10, max_no_improvement=10, verbose=0,
                      random_state=0),name="k-means++", data=data)
    return K

In [5]:
def compute_best_k(x,y,occurrencies, plot=False,points=1000,sf=0.9):
    import numpy as np
    
    if len(x)<5:
        return max(1, round(np.sqrt(occurrencies/2)))
    
    from scipy.interpolate import interp1d
    from scipy.interpolate import UnivariateSpline
    spl = UnivariateSpline(x, y)
    spl.set_smoothing_factor(sf)
    xs = np.linspace(min(x), max(x), points)
    ys = spl(xs)
    idx_better_k=get_change_point(xs, ys)
    if plot:
        import pylab
        pylab.plot(xs,ys)
        
        pylab.scatter(xs[idx_better_k],ys[idx_better_k],s=20, marker='o')
        pylab.text(xs[idx_better_k],ys[idx_better_k],"bestK %s" %(np.round(xs[idx_better_k])))
        return int(np.round(xs[idx_better_k])),pylab
    return int(np.round(xs[idx_better_k]))

In [6]:
def bench_k_means(estimator, name, data,distance_function=None):
    from sklearn import metrics
    from sklearn.metrics import silhouette_samples, silhouette_score
    import time
    t0 = time.time()
    if distance_function:
        estimator.fit(data,distance_function)
    else:
        estimator.fit(data)
    #cluster_labels = estimator.fit_predict(data)
    #silhouette_score_ = silhouette_score(data, cluster_labels)
    
    inertia=estimator.inertia_
    duration=time.time() - t0
    return {'inertia':inertia,'duration':duration, 'estimator':estimator}#,'silhouette':silhouette_score_}

def get_change_point(x, y):
    """
         Elección del mejor K
         :: param x: lista de valores de K
         :: param y: lista de valores de SSE
    """
    import math
    max_d = -float('infinity')
    index = 0

    for i in range(0, len(x)):
        c = closest_point_on_segment(a=[x[0], y[0]], b=[x[len(x)-1], y[len(y)-1]], p=[x[i], y[i]])
        d = math.sqrt((c[0]-x[i])**2 + (c[1]-y[i])**2)
        if d > max_d:
            max_d = d
            index = i
    
    return index

def closest_point_on_segment(a, b, p):
    sx1 = a[0]
    sx2 = b[0]
    sy1 = a[1]
    sy2 = b[1]
    px = p[0]
    py = p[1]

    x_delta = sx2 - sx1
    y_delta = sy2 - sy1

    if x_delta == 0 and y_delta == 0:
        return p

    u = ((px - sx1) * x_delta + (py - sy1) * y_delta) / (x_delta * x_delta + y_delta *  y_delta)
    if u < 0:
        closest_point = a
    elif u > 1:
        closest_point = b
    else:
        cp_x = sx1 + u * x_delta
        cp_y = sy1 + u * y_delta
        closest_point = [cp_x, cp_y]

    return closest_point
	

### Individual Clustering

In [7]:
# Numero de filas del archivo
f=open(individual_footprint)
num_rows = len(f.readlines())-1
f.close()

#<customer_id;year;week;profile_id;size;t1... tn >
import datetime
f=open(individual_footprint)
fw=open(individual_clusters,'w')  #uid,cluster_id,centroid
fw2=open(individual_labels,'w') #uid,year,week,cluster_id,profile
fw.write('customer_tag;individual_cluster;d0;d1;d2;d3;d4;d5;d6\n')
fw2.write('customer_tag;year;week;individual_cluster;d0;d1;d2;d3;d4;d5;d6\n')

f.readline()
data=[] #buffer

footprints_clustered=0
footprints_clusters=0
n_cliente=0
contador = 0
temporal= 0
for row in f: #reading individual footprint
    row=row.strip().split(',') # leemos cada elemento da linea parseada por ","
    uid=row[0]
    year=row[1]
    week=row[2]
    size=int(row[4])
    profile=np.array([float(el) for el in row[5:]])
    # Individual clustering
    if uid==clientes[n_cliente]: # Para cada fila donde los "uid" son iguales 
        data.append(((uid,year,week),profile))     
        contador+=1
    else: #final de cliente
        
        #---------------------------------------------------------------------
        # procesar data
        #---------------------------------------------------------------------
        to_cluster=[el[1] for el in data]
        K=process_footprint(to_cluster,np.arange(1,len(to_cluster)+1))

        # choose k
        x=list(K.keys())
        y=[K[k]['inertia'] for k in K]
        best_k=compute_best_k(x,y,len(to_cluster))
        print(str(contador)+' => clustering: '+str(clientes[n_cliente])+' len data: '+str(len(data))+" best k: "+str(best_k))
        
        # clustering
        if best_k==1:
            #to few records
            cluster_centers_=[np.average(to_cluster,axis=0)]
            labels_=[0]*len(to_cluster)  
        else:
            cluster_centers_=K[best_k]['estimator'].cluster_centers_
            labels_=K[best_k]['estimator'].labels_
        
        #export individual centroids
        for i in np.arange(len(cluster_centers_)):
            string="%s;%s;%s\n"%(clientes[n_cliente],i,';'.join([str(el) for el in cluster_centers_[i]])) #uid,cluster_id,centroid
            fw.write(string)
            footprints_clusters+=1
        fw.flush()

        #export original data and labels
        for i in np.arange(len(data)):
            uid2=data[i][0]
            profile2=data[i][1]
            label2=labels_[i]
            string="%s;%s;%s;%s;%s\n" %(uid2[0],uid2[1],uid2[2],label2
                                                    ,';'.join([str(el) for el in profile2]))#uid,year,week,cluster_id,profile
            fw2.write(string)
            footprints_clustered+=1
        fw2.flush()
        #---------------------------------------------------------------------
        #---------------------------------------------------------------------
        
        

        data=[] #buffer
        data.append(((uid,year,week),profile))
        
        contador+=1
        temporal+=1
        n_cliente+=1
        
    if contador == num_rows:        # Para el ultimo cliente y ultima fila
        #---------------------------------------------------------------------
        # procesar data
        #---------------------------------------------------------------------
        to_cluster=[el[1] for el in data]
        K=process_footprint(to_cluster,np.arange(1,len(to_cluster)+1))

        # choose k
        x=list(K.keys())
        y=[K[k]['inertia'] for k in K]
        best_k=compute_best_k(x,y,len(to_cluster))
        print(str(contador)+' => clustering: '+str(clientes[n_cliente])+' len data: '+str(len(data))+" best k: "+str(best_k))
        
        # clustering
        if best_k==1:
            #to few records
            cluster_centers_=[np.average(to_cluster,axis=0)]
            labels_=[0]*len(to_cluster)  
        else:
            cluster_centers_=K[best_k]['estimator'].cluster_centers_
            labels_=K[best_k]['estimator'].labels_
        
        #export individual centroids
        for i in np.arange(len(cluster_centers_)):
            string="%s;%s;%s\n"%(uid,i,';'.join([str(el) for el in cluster_centers_[i]])) #uid,cluster_id,centroid
            fw.write(string)
            footprints_clusters+=1
        fw.flush()

        #export original data and labels
        for i in np.arange(len(data)):
            uid2=data[i][0]
            profile2=data[i][1]
            label2=labels_[i]
            string="%s;%s;%s;%s;%s\n" %(uid2[0],uid2[1],uid2[2],label2
                                                    ,';'.join([str(el) for el in profile2]))#uid,year,week,cluster_id,profile
            fw2.write(string)
            footprints_clustered+=1
        fw2.flush()
        #---------------------------------------------------------------------
        #---------------------------------------------------------------------
        
        print("final")   
    
    

temporal

15 => clustering: +++g8j9k+5A= len data: 15 best k: 6
44 => clustering: ++/9crwdWSc= len data: 29 best k: 10
82 => clustering: ++/NRCUAMIA= len data: 38 best k: 9
83 => clustering: ++/bAm/9MuA= len data: 1 best k: 1
110 => clustering: ++/nqADqHkE= len data: 27 best k: 7
114 => clustering: ++/oQ9Lb9dI= len data: 4 best k: 1
118 => clustering: ++0i87zUaWQ= len data: 4 best k: 1
153 => clustering: ++0xinBYSBE= len data: 35 best k: 8
157 => clustering: ++14g8obpj0= len data: 4 best k: 1
159 => clustering: ++1QchhSBuo= len data: 2 best k: 1
200 => clustering: ++1XMtcwMec= len data: 41 best k: 10
203 => clustering: ++27tu/7+9g= len data: 3 best k: 1
204 => clustering: ++2AgIYFkQ8= len data: 1 best k: 1
211 => clustering: ++2IlnOb058= len data: 7 best k: 4
219 => clustering: ++2bJYMARJk= len data: 8 best k: 4
222 => clustering: ++2fs4nrZPo= len data: 3 best k: 1
223 => clustering: ++2rVK+/tXs= len data: 1 best k: 1
225 => clustering: ++3AMD1kLjA= len data: 2 best k: 1
265 => clustering: ++3P3

1929 => clustering: ++ZZAXWCx9U= len data: 30 best k: 9
1938 => clustering: ++aBU5hNvP4= len data: 9 best k: 5
1988 => clustering: ++aHRvQdI2E= len data: 50 best k: 11
2003 => clustering: ++aPqiCUKXU= len data: 15 best k: 1
2027 => clustering: ++aY+RmK3rQ= len data: 24 best k: 7
2031 => clustering: ++apsUzdF/A= len data: 4 best k: 1
2050 => clustering: ++avHOqcQZg= len data: 19 best k: 9
2051 => clustering: ++bCU4cREQQ= len data: 1 best k: 1
2055 => clustering: ++bkjQ+iDNs= len data: 4 best k: 1
2056 => clustering: ++bsxpSJRP4= len data: 1 best k: 1
2064 => clustering: ++clOnxbpk0= len data: 8 best k: 4
2089 => clustering: ++cqWmONHM8= len data: 25 best k: 8
2102 => clustering: ++cwkQT9HuM= len data: 13 best k: 6
2106 => clustering: ++dAyWOTXAk= len data: 4 best k: 1
2146 => clustering: ++dXyeMZnRg= len data: 40 best k: 10
2152 => clustering: ++dYNgqHClE= len data: 6 best k: 2
2154 => clustering: ++dx0F6J4n4= len data: 2 best k: 1
2161 => clustering: ++eZlrJULbs= len data: 7 best k: 3


3731 => clustering: +/1N6bac2+8= len data: 48 best k: 9
3777 => clustering: +/1bIPzKF/E= len data: 46 best k: 11
3784 => clustering: +/1lGuf8S6Q= len data: 7 best k: 3
3785 => clustering: +/1o3XvOZ58= len data: 1 best k: 1
3787 => clustering: +/1r3dQnFwk= len data: 2 best k: 1
3791 => clustering: +/1wNGlI4L8= len data: 4 best k: 1
3795 => clustering: +/1xvnel1fw= len data: 4 best k: 1
3799 => clustering: +/3ZCyTGE/A= len data: 4 best k: 1
3806 => clustering: +/3aJjVJVnk= len data: 7 best k: 4
3808 => clustering: +/3pqgvgCzo= len data: 2 best k: 1
3811 => clustering: +/4/FGyY3Ec= len data: 3 best k: 1
3812 => clustering: +/47YwGr248= len data: 1 best k: 1
3860 => clustering: +/5Eo2IAr/8= len data: 48 best k: 13
3863 => clustering: +/5NjtcPsXQ= len data: 3 best k: 1
3864 => clustering: +/5d6MEMdyg= len data: 1 best k: 1
3865 => clustering: +/6MnR7KK/k= len data: 1 best k: 1
3872 => clustering: +/6oMKc5E7g= len data: 7 best k: 3
3874 => clustering: +/6pFkRKLOg= len data: 2 best k: 1
3879 

5435 => clustering: +/f8o9ONOow= len data: 15 best k: 6
5439 => clustering: +/fJ/IRbkUA= len data: 4 best k: 1
5443 => clustering: +/g9ZV6ROPc= len data: 4 best k: 1
5444 => clustering: +/gIj7J6/qY= len data: 1 best k: 1
5482 => clustering: +/gXibKHs04= len data: 38 best k: 9
5485 => clustering: +/gibTt07O0= len data: 3 best k: 1
5531 => clustering: +/gzDLGsDLY= len data: 46 best k: 12
5546 => clustering: +/gzkeNhhz4= len data: 15 best k: 5
5556 => clustering: +/h6HlctOvc= len data: 10 best k: 4
5557 => clustering: +/hPSWKEJYo= len data: 1 best k: 1
5580 => clustering: +/hbYzZV/Es= len data: 23 best k: 8
5584 => clustering: +/hoWSU9ODk= len data: 4 best k: 1
5629 => clustering: +/hxtu5mmSk= len data: 45 best k: 11
5669 => clustering: +/i0RR6Srn8= len data: 40 best k: 9
5700 => clustering: +/iI+4Xnqb0= len data: 31 best k: 7
5702 => clustering: +/iWK36Jm5A= len data: 2 best k: 1
5705 => clustering: +/ifHandHpw= len data: 3 best k: 1
5729 => clustering: +/irfhG5ElQ= len data: 24 best k: 

7076 => clustering: +09L2820aF8= len data: 14 best k: 5
7084 => clustering: +0A8y+Hw8rE= len data: 8 best k: 4
7089 => clustering: +0AHVKegd9Q= len data: 5 best k: 2
7092 => clustering: +0AathEAnOY= len data: 3 best k: 1
7097 => clustering: +0BcaL6KLp8= len data: 5 best k: 3
7104 => clustering: +0BoaWlaXZE= len data: 7 best k: 4
7106 => clustering: +0CJwVV6VTg= len data: 2 best k: 1
7107 => clustering: +0CLTbo8kEQ= len data: 1 best k: 1
7108 => clustering: +0CTOl/wIwE= len data: 1 best k: 1
7122 => clustering: +0CdF34H9j8= len data: 14 best k: 7
7128 => clustering: +0CwZYdJtik= len data: 6 best k: 3
7173 => clustering: +0CyBDzgj3c= len data: 45 best k: 13
7209 => clustering: +0D87UEt3fs= len data: 36 best k: 8
7212 => clustering: +0DB/9wocrc= len data: 3 best k: 1
7213 => clustering: +0DEfQja2Pg= len data: 1 best k: 1
7214 => clustering: +0DOzp/OCuI= len data: 1 best k: 1
7217 => clustering: +0DXTuodmrM= len data: 3 best k: 1
7231 => clustering: +0DYz/dowGk= len data: 14 best k: 6
7242

...

372978 => clustering: /j9uPaf7IEU= len data: 5 best k: 3
372980 => clustering: /jA2s6FUoIs= len data: 2 best k: 1
372983 => clustering: /jA4h9nQ2H4= len data: 3 best k: 1
372985 => clustering: /jA6AaQwGyQ= len data: 2 best k: 1
372986 => clustering: /jAE5bPLUuE= len data: 1 best k: 1
372988 => clustering: /jAHXwNXDww= len data: 2 best k: 1
373013 => clustering: /jAMdd72i+0= len data: 25 best k: 8
373041 => clustering: /jAr5hRPgbw= len data: 28 best k: 8
373042 => clustering: /jBzdw5nFJM= len data: 1 best k: 1
373044 => clustering: /jC8osPW644= len data: 2 best k: 1
373059 => clustering: /jCdNheqVSw= len data: 15 best k: 6
373068 => clustering: /jCw8T2GnYE= len data: 9 best k: 4
373083 => clustering: /jDI+gRnwZQ= len data: 15 best k: 1
373118 => clustering: /jDNj2HioRM= len data: 35 best k: 8
373123 => clustering: /jDe4HpZIjg= len data: 5 best k: 3
373142 => clustering: /jDhzUtldr8= len data: 19 best k: 6
373150 => clustering: /jDyiEdeJuM= len data: 8 best k: 4
373153 => clustering: /jE

374700 => clustering: /jjIKzSRfDQ= len data: 39 best k: 11
374701 => clustering: /jjnUVsGOHE= len data: 1 best k: 1
374721 => clustering: /jjwHL5ZXp4= len data: 20 best k: 8
374759 => clustering: /jjxhqKRv0g= len data: 38 best k: 1
374781 => clustering: /jjzD+SZYbM= len data: 22 best k: 5
374810 => clustering: /jkHf4EpAaQ= len data: 29 best k: 7
374827 => clustering: /jkLi86Qt7A= len data: 17 best k: 7
374879 => clustering: /jkfkyaIz1A= len data: 52 best k: 13
374880 => clustering: /jlA5t8mqng= len data: 1 best k: 1
374882 => clustering: /jlgWJxoT2I= len data: 2 best k: 1
374892 => clustering: /jm2YCuyX18= len data: 10 best k: 5
374902 => clustering: /jm9rwB67Xk= len data: 10 best k: 5
374905 => clustering: /jmKuZC1YaU= len data: 3 best k: 1
374908 => clustering: /jmZSnyVM5o= len data: 3 best k: 1
374911 => clustering: /jmcgwZh64g= len data: 3 best k: 1
374933 => clustering: /jmh0bst+aU= len data: 22 best k: 6
374934 => clustering: /jnIESAAJlk= len data: 1 best k: 1
374935 => clusterin

376534 => clustering: /kD7c0CfHug= len data: 39 best k: 9
376537 => clustering: /kDAtiVYZuI= len data: 3 best k: 1
376543 => clustering: /kDIUpywH9s= len data: 6 best k: 3
376567 => clustering: /kDNcCC7T7I= len data: 24 best k: 7
376569 => clustering: /kDcFJpMjDA= len data: 2 best k: 1
376603 => clustering: /kEdfLVvbCE= len data: 34 best k: 13
376617 => clustering: /kEfp4E7zC4= len data: 14 best k: 4
376619 => clustering: /kFRexKamfo= len data: 2 best k: 1
376634 => clustering: /kFeZSR8HFc= len data: 15 best k: 6
376656 => clustering: /kFzB9Z6hWc= len data: 22 best k: 7
376658 => clustering: /kFzIZsCREw= len data: 2 best k: 1
376708 => clustering: /kG5cYqeFro= len data: 50 best k: 1
376712 => clustering: /kGcwB9J6kk= len data: 4 best k: 1
376719 => clustering: /kHMsTaDIA0= len data: 7 best k: 3
376741 => clustering: /kHgxkuM4Wg= len data: 22 best k: 6
376754 => clustering: /kI0oNnRFoI= len data: 13 best k: 5
376762 => clustering: /kI8/bQZXDU= len data: 8 best k: 5
376778 => clustering:

378310 => clustering: /kk46BNiQyo= len data: 27 best k: 7
378331 => clustering: /kkCuJmMyh0= len data: 21 best k: 8
378339 => clustering: /kkHjOUSuwA= len data: 8 best k: 4
378340 => clustering: /kkJ4xTZA90= len data: 1 best k: 1
378342 => clustering: /kkZG8t861Q= len data: 2 best k: 1
378346 => clustering: /kkets8mYds= len data: 4 best k: 1
378367 => clustering: /kkjXGHTdGc= len data: 21 best k: 8
378398 => clustering: /kklO1hidNI= len data: 31 best k: 7
378407 => clustering: /kkxTv5wj5Y= len data: 9 best k: 4
378409 => clustering: /klEoKqpAcY= len data: 2 best k: 1
378410 => clustering: /km4GQSG+tM= len data: 1 best k: 1
378435 => clustering: /kmSvXbXYgc= len data: 25 best k: 8
378440 => clustering: /kmXuVOtWEM= len data: 5 best k: 3
378449 => clustering: /kmZpQSUzaA= len data: 9 best k: 5
378450 => clustering: /kmksZzrZq4= len data: 1 best k: 1
378476 => clustering: /kmn8XEoyuk= len data: 26 best k: 8
378483 => clustering: /kn1/tBCOPg= len data: 7 best k: 3
378490 => clustering: /kn

31557